In [16]:
import pandas as pd
import itertools

# Nombre del archivo CSV local
csv_file_name = 'historico_ajustes_para_relacion.csv'

# Función principal
def procesar_spread_localmente():
    try:
        # Leer el archivo CSV existente localmente
        try:
            combined_data = pd.read_csv(csv_file_name)
            if 'FECHA' not in combined_data.columns:
                raise ValueError("La columna 'FECHA' no está presente en el archivo CSV.")
        except FileNotFoundError:
            combined_data = pd.DataFrame(columns=["PRODUCTO", "TIPO CONTRATO", "AJUSTE / PRIMA REF.", "AÑO", "MES-DIA", "FECHA"])
            return "No se encontró el archivo CSV local. Se ha creado un DataFrame vacío."

        # Asegurarse de que la columna 'FECHA' esté en formato datetime
        combined_data['FECHA'] = pd.to_datetime(combined_data['FECHA'], format='%Y-%m-%d', errors='coerce')
        
        # Generar una lista de tuplas que contenga todas las combinaciones de productos y posiciones
        productos_posiciones = combined_data[['PRODUCTO', 'TIPO CONTRATO']].drop_duplicates().values.tolist()

        # Generar todas las combinaciones de productos y posiciones, incluyendo comparaciones invertidas
        combinaciones = [(p1, t1, p2, t2) for (p1, t1), (p2, t2) in itertools.product(productos_posiciones, repeat=2)
                         if not (p1 == p2 and t1 == t2)]  # Excluir comparaciones con el mismo producto y posición

        resultados = []
        for p1, t1, p2, t2 in combinaciones:
            df_pos1 = combined_data[(combined_data['PRODUCTO'] == p1) & (combined_data['TIPO CONTRATO'] == t1)]
            df_pos2 = combined_data[(combined_data['PRODUCTO'] == p2) & (combined_data['TIPO CONTRATO'] == t2)]

            # Utilizar la columna 'FECHA' para hacer la combinación
            merged = pd.merge(df_pos1, df_pos2, on='FECHA', suffixes=('_pos1', '_pos2'))
            
            merged['SPREAD'] = (merged['AJUSTE / PRIMA REF._pos1'] - merged['AJUSTE / PRIMA REF._pos2']).round(3)
            merged['RELACION%'] = ((merged['AJUSTE / PRIMA REF._pos1'] / merged['AJUSTE / PRIMA REF._pos2'] - 1) * 100).round(3)

            for index, row in merged.iterrows():
                resultados.append({
                    'FECHA': row['FECHA'].strftime('%Y-%m-%d'),  # Usamos la fecha generada
                    'PRODUCTO_1': p1,
                    'TIPO_CONTRATO_1': t1,
                    'PRODUCTO_2': p2,
                    'TIPO_CONTRATO_2': t2,
                    'AJUSTE_POS1': row['AJUSTE / PRIMA REF._pos1'],
                    'AJUSTE_POS2': row['AJUSTE / PRIMA REF._pos2'],
                    'SPREAD': row['SPREAD'],
                    'RELACION%': row['RELACION%']
                })

        # Guardar los resultados en un nuevo archivo CSV local
        resultados_df = pd.DataFrame(resultados)
        resultados_df.to_csv('historico_relacion.csv', index=False)

        # Contar cuántas filas se guardaron en spread_comparison.csv
        filas_spread_comparison = len(resultados_df)

        return f"Proceso completado. Se guardaron {filas_spread_comparison} filas en 'historico_relacion.csv'."

    except Exception as e:
        return f"Error durante el procesamiento: {str(e)}"

# Ejecutar la función
resultado = procesar_spread_localmente()
print(resultado)

Proceso completado. Se guardaron 235642 filas en 'historico_comparacion.csv'.


In [3]:
import pandas as pd
import itertools
import requests
import csv
import re
from datetime import datetime
import json

# Nombre del archivo CSV local
csv_file_name = 'historico_ajustes_para_relacion.csv'

def obtener_productos_agricolas():
    try:
        # Leer el archivo CSV existente localmente
        with open(csv_file_name, mode='r', encoding='utf-8') as file:
            csv_reader = csv.DictReader(file)
            productos_agricolas = set(row['PRODUCTO'] for row in csv_reader)
        return productos_agricolas
    except FileNotFoundError:
        return set()

def es_opcion(tipo_contrato):
    # Patrón que detecta si el contrato contiene un número seguido de una letra (p.ej. "185 C")
    patron_opcion = re.compile(r'\d+ [A-Z]')
    return bool(patron_opcion.search(tipo_contrato))

def procesar_spread_localmente():
    try:
        # Calcular la fecha de hoy
        today = datetime.now()
        today_str = today.strftime('%Y-%m-%d')

        # Definir la URL del endpoint de precios de cierre
        url_closing_prices = 'https://apicem.matbarofex.com.ar/api/v2/closing-prices'
        params = {
            "from": today_str, #"2024-08-06",
            "to": today_str, #"2024-08-06",
            "market": "ROFX",
            "version": "v2"
        }

        # Obtener la lista de productos agrícolas
        productos_agricolas = obtener_productos_agricolas()

        # Realizar la solicitud a la API para obtener la lista de precios de cierre
        response_closing_prices = requests.get(url_closing_prices, params=params)
        
        if response_closing_prices.status_code != 200:
            return f"Error en la solicitud a la API: {response_closing_prices.status_code}"

        try:
            data_closing_prices = response_closing_prices.json()

            # Filtrar las columnas deseadas y cambiar el formato de los nombres de las columnas
            filtered_data = [
                {
                    "FECHA": item.get("dateTime")[:10],  # Extraer solo la fecha
                    "PRODUCTO": item.get("product"),
                    "TIPO CONTRATO": item.get("symbol"),
                    "AJUSTE / PRIMA REF.": item.get("settlement")
                }
                for item in data_closing_prices.get('data', [])
                if item.get("product") in productos_agricolas and not es_opcion(item.get("symbol"))
            ]

            # # Cargar los datos existentes si el archivo CSV existe
            # try:
            #     existing_data = pd.read_csv(csv_file_name)
            # except FileNotFoundError:
            #     existing_data = pd.DataFrame(columns=["FECHA", "PRODUCTO", "TIPO CONTRATO", "AJUSTE / PRIMA REF."])

            # # Combinar los datos existentes con los nuevos datos
            new_data = pd.DataFrame(filtered_data)

            # Asegurarse de que la columna 'FECHA' esté en formato datetime
            new_data['FECHA'] = pd.to_datetime(new_data['FECHA'], format='%Y-%m-%d', errors='coerce')

            # combined_data = pd.concat([existing_data, new_data]).drop_duplicates().reset_index(drop=True)

            # # Asegurarse de que la columna 'FECHA' esté en formato datetime
            # combined_data['FECHA'] = pd.to_datetime(combined_data['FECHA'], format='%Y-%m-%d', errors='coerce')

            # # Guardar los datos combinados en el archivo CSV
            # combined_data.to_csv(csv_file_name, index=False)

            # Generar una lista de tuplas que contenga todas las combinaciones de productos y posiciones
            productos_posiciones = new_data[['PRODUCTO', 'TIPO CONTRATO']].drop_duplicates().values.tolist()

            # Generar todas las combinaciones de productos y posiciones, incluyendo comparaciones invertidas
            combinaciones = [(p1, t1, p2, t2) for (p1, t1), (p2, t2) in itertools.product(productos_posiciones, repeat=2)
                             if not (p1 == p2 and t1 == t2)]  # Excluir comparaciones con el mismo producto y posición

            resultados = []
            for p1, t1, p2, t2 in combinaciones:
                df_pos1 = new_data[(new_data['PRODUCTO'] == p1) & (new_data['TIPO CONTRATO'] == t1)]
                df_pos2 = new_data[(new_data['PRODUCTO'] == p2) & (new_data['TIPO CONTRATO'] == t2)]

                # Utilizar la columna 'FECHA' para hacer la combinación
                merged = pd.merge(df_pos1, df_pos2, on='FECHA', suffixes=('_pos1', '_pos2'))

                merged['SPREAD'] = (merged['AJUSTE / PRIMA REF._pos1'] - merged['AJUSTE / PRIMA REF._pos2']).round(3)
                merged['RELACION%'] = ((merged['AJUSTE / PRIMA REF._pos1'] / merged['AJUSTE / PRIMA REF._pos2'] - 1) * 100).round(3)

                for index, row in merged.iterrows():
                    resultados.append({
                        'FECHA': row['FECHA'].strftime('%Y-%m-%d'),  # Usamos la fecha generada
                        'PRODUCTO_1': p1,
                        'TIPO_CONTRATO_1': t1,
                        'PRODUCTO_2': p2,
                        'TIPO_CONTRATO_2': t2,
                        'AJUSTE_POS1': row['AJUSTE / PRIMA REF._pos1'],
                        'AJUSTE_POS2': row['AJUSTE / PRIMA REF._pos2'],
                        'SPREAD': row['SPREAD'],
                        'RELACION%': row['RELACION%']
                    })

            # Guardar los resultados en un nuevo archivo CSV local
            resultados_df = pd.DataFrame(resultados)
            resultados_df.to_csv('relaciones_hoy.csv', index=False)

            filas_spread_comparison = len(resultados_df)
            return f"Proceso completado. Se guardaron {filas_spread_comparison} filas en 'historico_relacion.csv'."

        except json.JSONDecodeError:
            return "Error al decodificar la respuesta de la API."

    except Exception as e:
        return f"Error durante el procesamiento: {str(e)}"

# Ejecutar la función
resultado = procesar_spread_localmente()
print(resultado)

Proceso completado. Se guardaron 240 filas en 'historico_comparacion.csv'.


In [6]:
import pandas as pd
import itertools
import requests
import csv
import re
from datetime import datetime
import json

# Nombre del archivo CSV local
csv_file_name = 'historico_ajustes_para_relacion.csv'

def obtener_productos_agricolas():
    try:
        # Leer el archivo CSV existente localmente
        with open(csv_file_name, mode='r', encoding='utf-8') as file:
            csv_reader = csv.DictReader(file)
            productos_agricolas = set(row['PRODUCTO'] for row in csv_reader)
        return productos_agricolas
    except FileNotFoundError:
        return set()

def es_opcion(tipo_contrato):
    # Patrón que detecta si el contrato contiene un número seguido de una letra (p.ej. "185 C")
    patron_opcion = re.compile(r'\d+ [A-Z]')
    return bool(patron_opcion.search(tipo_contrato))

def procesar_spread_localmente():
    try:
        # Calcular la fecha de hoy
        today = datetime.now()
        today_str = today.strftime('%Y-%m-%d')

        # Definir la URL del endpoint de precios de cierre
        url_closing_prices = 'https://apicem.matbarofex.com.ar/api/v2/closing-prices'
        params = {
            "from": "2024-08-06",
            "to": "2024-08-06",
            "market": "ROFX",
            "version": "v2"
        }

        # Obtener la lista de productos agrícolas
        productos_agricolas = obtener_productos_agricolas()

        # Realizar la solicitud a la API para obtener la lista de precios de cierre
        response_closing_prices = requests.get(url_closing_prices, params=params)
        
        if response_closing_prices.status_code != 200:
            return f"Error en la solicitud a la API: {response_closing_prices.status_code}"

        try:
            data_closing_prices = response_closing_prices.json()

            # Filtrar las columnas deseadas y cambiar el formato de los nombres de las columnas
            filtered_data = [
                {
                    "FECHA": item.get("dateTime")[:10],  # Extraer solo la fecha
                    "PRODUCTO": item.get("product"),
                    "TIPO CONTRATO": item.get("symbol"),
                    "AJUSTE / PRIMA REF.": item.get("settlement")
                }
                for item in data_closing_prices.get('data', [])
                if item.get("product") in productos_agricolas and not es_opcion(item.get("symbol"))
            ]

            # Cargar los datos existentes si el archivo CSV existe
            try:
                existing_data = pd.read_csv(csv_file_name)
            except FileNotFoundError:
                existing_data = pd.DataFrame(columns=["FECHA", "PRODUCTO", "TIPO CONTRATO", "AJUSTE / PRIMA REF."])

            # # Combinar los datos existentes con los nuevos datos
            new_data = pd.DataFrame(filtered_data)

            # Asegurarse de que la columna 'FECHA' esté en formato datetime
            new_data['FECHA'] = pd.to_datetime(new_data['FECHA'], format='%Y-%m-%d', errors='coerce')

            combined_data = pd.concat([existing_data, new_data]).drop_duplicates().reset_index(drop=True)

            # Asegurarse de que la columna 'FECHA' esté en formato datetime
            combined_data['FECHA'] = pd.to_datetime(combined_data['FECHA'], format='%Y-%m-%d', errors='coerce')

            # Guardar los datos combinados en el archivo CSV
            combined_data.to_csv(csv_file_name, index=False)

            # Generar una lista de tuplas que contenga todas las combinaciones de productos y posiciones
            productos_posiciones = new_data[['PRODUCTO', 'TIPO CONTRATO']].drop_duplicates().values.tolist()

            # Generar todas las combinaciones de productos y posiciones, incluyendo comparaciones invertidas
            combinaciones = [(p1, t1, p2, t2) for (p1, t1), (p2, t2) in itertools.product(productos_posiciones, repeat=2)
                             if not (p1 == p2 and t1 == t2)]  # Excluir comparaciones con el mismo producto y posición

            resultados = []
            for p1, t1, p2, t2 in combinaciones:
                df_pos1 = new_data[(new_data['PRODUCTO'] == p1) & (new_data['TIPO CONTRATO'] == t1)]
                df_pos2 = new_data[(new_data['PRODUCTO'] == p2) & (new_data['TIPO CONTRATO'] == t2)]

                # Utilizar la columna 'FECHA' para hacer la combinación
                merged = pd.merge(df_pos1, df_pos2, on='FECHA', suffixes=('_pos1', '_pos2'))

                merged['SPREAD'] = (merged['AJUSTE / PRIMA REF._pos1'] - merged['AJUSTE / PRIMA REF._pos2']).round(3)
                merged['RELACION%'] = ((merged['AJUSTE / PRIMA REF._pos1'] / merged['AJUSTE / PRIMA REF._pos2'] - 1) * 100).round(3)

                for index, row in merged.iterrows():
                    resultados.append({
                        'FECHA': row['FECHA'].strftime('%Y-%m-%d'),  # Usamos la fecha generada
                        'PRODUCTO_1': p1,
                        'TIPO_CONTRATO_1': t1,
                        'PRODUCTO_2': p2,
                        'TIPO_CONTRATO_2': t2,
                        'AJUSTE_POS1': row['AJUSTE / PRIMA REF._pos1'],
                        'AJUSTE_POS2': row['AJUSTE / PRIMA REF._pos2'],
                        'SPREAD': row['SPREAD'],
                        'RELACION%': row['RELACION%']
                    })

            # Guardar los resultados en un nuevo archivo CSV local
            resultados_df = pd.DataFrame(resultados)
            resultados_df.to_csv('relaciones_hoy.csv', index=False)

            filas_spread_comparison = len(resultados_df)
            return f"Proceso completado. Se guardaron {filas_spread_comparison} filas en 'relaciones_hoy.csv'."

        except json.JSONDecodeError:
            return "Error al decodificar la respuesta de la API."

    except Exception as e:
        return f"Error durante el procesamiento: {str(e)}"

# Ejecutar la función
resultado = procesar_spread_localmente()
print(resultado)

Proceso completado. Se guardaron 240 filas en 'historico_comparacion.csv'.
